## Style-specific T2I Generation with Flux.1-dev

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from accelerate.utils import set_seed
from mypipeline import AttDistPipeline
from utils import *


model_name = "/root/models/FLUX.1-dev"
lr = 0.01
iters = 2
seed = 42
width = 512
height = 512
mixed_precision = "bf16"
num_inference_steps = 50
guidance_scale = 3.5
enable_gradient_checkpoint = True
start_layer, end_layer = 50, 57
start_time = 9999
prompt="A panda"


pipe = AttDistPipeline.from_pretrained(
    model_name, torch_dtype=torch.float16)


memory_efficient(pipe)
set_seed(seed)
loss_fn = torch.nn.L1Loss()

style_image = ["..//data/style/1.jpg"]
style_image = torch.cat([load_image(path, size=(512, 512)) for path in style_image])


controller = Controller(self_layers=(start_layer, end_layer))

result = pipe.sample(
    lr=lr,
    prompt=prompt,
    loss_fn=loss_fn,
    iters=iters,
    width=width,
    height=height,
    start_time=start_time,
    controller=controller,
    style_image=style_image,
    guidance_scale=guidance_scale,
    mixed_precision=mixed_precision,
    num_inference_steps=num_inference_steps,
    enable_gradient_checkpoint=enable_gradient_checkpoint,
)

save_image(style_image, "style.png")
save_image(result, "output.png")
show_image("style.png", title="style image")
show_image("output.png", title=prompt)